# Projeto 1 - Ciência dos Dados

Nome: Lourenço Meirelles

Nome: Lucca Lima

Nome: Pedro Gomes de Sá Drumond

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Pedro Drumond\Documents\Insper 2o semestre\CDados\GitHub_CD\Projeto_1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Primevideo.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao
0,"""eu sou mais eu"" não tem roteiro criativo, par...",2
1,#nineperfectstrangers \nsérie muito boa no @pr...,3
2,#primevideo quero uma sequência pra ontem de ...,2
3,@camila_cabello @cinderella @primevideo amei p...,3
4,@camila_cabello @cinderella @primevideo bravo!👏,3


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,@janainadobrasil esse daqui é a melhor opção p...,1
1,"@camila_cabello @primevideo ficou lindo, te amo",1
2,@camila_cabello @cinderella @primevideo linda!...,1
3,@camila_cabello @cinderella @primevideo ta lin...,1
4,@camilizecabello @mannytsakanika @cinderella @...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O Amazon Prime Video é uma plataforma de streaming semelhante à Netflix, na qual você pode acessar um catálogo de filmes, séries, etc... além de disponibilizar outras coisas como o Amazon Prime Music e entre outros.

Para classificar, consideramos mensagens que elogiassem a plataforma como relevante, e o resto como irrelevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def cleanup(text):
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    #MELHORAR
    
#tirar '@'
#talvez 'e', 'que'

In [7]:
#PRECISAMOS DE:

#P(mensagens relevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como irrelevantes)
#P(mensagens relevantes | classificadas como irrelevantes)
#Acurácia ?

In [8]:
#DAR UMA OLHADA EM PANDAS
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
primevideo = pd.read_excel('Primevideo.xlsx')
primevideo.Classificacao = primevideo.Classificacao.astype('int')
primevideo

,Treinamento,Classificacao
0,"""eu sou mais eu"" não tem roteiro criativo, par...",2
1,#nineperfectstrangers \nsérie muito boa no @pr...,3
2,#primevideo quero uma sequência pra ontem de ...,2
3,@camila_cabello @cinderella @primevideo amei p...,3
4,@camila_cabello @cinderella @primevideo bravo!👏,3
...,...,...
494,"segundo a plataforma justwatch, o telecine tem...",1
495,deve q eu achei q tava rica qd assinei spotify...,1
496,"@acordadebora um crime real, dois filmes e dua...",0
497,quebrando a cabeça pra saber se a dubladora de...,1


In [9]:
#Criar um DF (DataFrame) para cada classificação

#Filtra as linhas, separando cada clasificação em um DF sem ter aplicado o 'CleanUp'
filtra_linhas = primevideo.Classificacao == 0
primevideo_classfic_0_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

filtra_linhas = primevideo.Classificacao == 1
primevideo_classfic_1_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 2
primevideo_classfic_2_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 3
primevideo_classfic_3_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

print(primevideo_classfic_0_raw)

0      @artofesthetic baseado no caso ritchofen, dois...
1      @laressa baseado no caso ritchofen, dois filme...
2      @popzonebr baseado no caso ritchofen, dois fil...
3      @bbb @gshow baseado no caso ritchofen, dois fi...
4      @flamengo @davidluiz_4 baseado no caso ritchof...
                             ...                        
142    @paulaamorim baseado no caso ritchofen, dois f...
143    @eulaurakeller @valescaoficial baseado no caso...
144    @sepcarloslima @acordadebora um crime real, do...
145    @acordadebora um crime real, dois filmes e dua...
146    @rolealeatorio baseado no caso ritchofen, dois...
Name: Treinamento, Length: 147, dtype: object


In [10]:
#CLEANUP - vai precisar do for
#cleanup(primevideo_classfic_0_raw[0].lower()).split()

In [30]:
#For, criando as listas limpas
lista_primevideo_classfic_0 = []
lista_primevideo_classfic_1 = []
lista_primevideo_classfic_2 = []
lista_primevideo_classfic_3 = []

lista_com_todos_tuites_limpos = []

#Cria a lista lista_primevideo_classfic que possui  as classificações de cada categoria

for celula in primevideo_classfic_0_raw:
    
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa.split()
    lista_primevideo_classfic_0 += celula_limpa.split()
    
    lista_tuites = []
    lista_tuites.append(celula_limpa.split()) #cria uma lista com todos os tuites splitados, tá duplicando. Queremos só adicionar cada tuite como se fosse uma lista
    #print(lista_primevideo_classfic_0)
    
for celula in primevideo_classfic_1_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa.split()
    lista_primevideo_classfic_1 += celula_limpa.split()

    lista_tuites = []
    lista_tuites.append(celula_limpa.split())
    print(lista_tuites)
    
for celula in primevideo_classfic_2_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa.split()
    lista_primevideo_classfic_2 += celula_limpa.split()
    
    lista_tuites = []
    lista_tuites.append(celula_limpa.split())
    #print(lista_primevideo_classfic_2)
    
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa.split()
    lista_primevideo_classfic_3 += celula_limpa.split()
    
    lista_tuites = []
    lista_tuites.append(celula_limpa.split())
    #print(lista_primevideo_classfic_3)

    
    
'''for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower()).split()
    lista_primevideo_classfic_3 += celula_limpa
    print(lista_primevideo_classfic_3)'''

[['alguém', 'me', 'empresta', 'o', 'primevideo', 'por', 'favor', 'eu', 'quero', 'ver', 'o', 'filme', 'da', 'cinderela', 'com', 'a', 'minha', 'mãe']]
[['assinei', 'o', 'primevideo', 'e', 'vou', 'finalmente', 'assistir', 'o', 'filme', 'da', 'camila', 'aaaaaaaaaa']]
[['gnt', 'alguem', 'tem', 'primevideo', 'pra', 'me', 'emprestar', 'quero', 'mt', 'assistir', 'um', 'filme😔']]
[['o', 'pico', 'da', 'comédia', 'foi', 'bem', 'aqui', 'https//tco/dmxo9porlh', 'https//tco/5bnmn8wldq']]
[['joguei', 'duas', 'partidas', 'hj', 'pq', 'fazia', 'tempo', 'sem', 'jogar', 'com', 'ela', 'ai', 'vem', 'skin', 'dela', 'no', 'prime', 'prime', 'nunca', 'falhou', 'comigo', 'https//tco/s3zjjwkdce']]
[['bayonetta', 'na', 'prime', 'vídeo', 'pra', 'começar', 'o', 'final', 'de', 'semana', 'com', 'o', 'pé', 'direito', 'que', 'tal', 'já', 'ir', 'pensando', 'em', 'um', 'bom', 'filme', 'de', 'anime', 'pra', 'curtir', 'temos', 'uma', 'ótima', 'sugestão', 'bayonetta', 'destino', 'sangrento', 'filme', 'sensacional', 'para', '

'for celula in primevideo_classfic_3_raw:\n    celula_limpa = cleanup(celula.lower()).split()\n    lista_primevideo_classfic_3 += celula_limpa\n    print(lista_primevideo_classfic_3)'

In [31]:
#Convertendo as tabelas para pd.Series
classfic_0 = pd.Series(lista_primevideo_classfic_0)
classfic_1 = pd.Series(lista_primevideo_classfic_1)
classfic_2 = pd.Series(lista_primevideo_classfic_2)
classfic_3 = pd.Series(lista_primevideo_classfic_3)

In [41]:
#Tabelas de contagem de ocorrencias em cada tabela, usando o value_counts()
tabela_classific_0 = classfic_0.value_counts()
tabela_classific_1 = classfic_1.value_counts()
tabela_classific_2 = classfic_2.value_counts()
tabela_classific_3 = classfic_3.value_counts() #TALVEZ COLOCAR FREQUÊNCIA RELATIVA

#Juntando todos os tuítes
todos_tuites = lista_primevideo_classfic_0 + lista_primevideo_classfic_1 + lista_primevideo_classfic_2 + lista_primevideo_classfic_3
#Criando tabela dos tuítes usando o pd.Series
todos_tuites_lista = pd.Series(todos_tuites)
tabela_todos_tuites = todos_tuites_lista.value_counts()

#ACHO QUE VAMOS TER QUE USAR TABELA RELATIVA PORQUE TEMOS MAIS AVALIAÇÕES EM UNS DO QUE OUTROS, VER ISSO!

a                        42
e                        32
o                        31
prime                    28
@primevideo              28
                         ..
https//tco/dx3mddi0fz     1
https//tco/u5btcfwrnn     1
bonitinho                 1
ligo                      1
grave                     1
Length: 572, dtype: int64


    Se P(0|tuite) > P(1|tuite) e P(0|tuite) > P(2|tuite) e P(0|tuite) > P(3|tuite):
        é 0
    Se P(1|tuite) > P(0|tuite) e P(1|tuite) > P(2|tuite) e P(1|tuite) > P(3|tuite):
        é 1
    Se P(2|tuite) > P(0|tuite) e P(2|tuite) > P(1|tuite) e P(2|tuite) > P(3|tuite):
        é 2
    Se P(3|tuite) > P(0|tuite) e P(3|tuite) > P(1|tuite) e P(3|tuite) > P(2|tuite):
        é e

In [45]:
#Começando classificador:

# P_0_dado_tuite = (P_tuite_dado_0 * P_0) / P_tuite

#acessa cada tuíte feito
P_0_dado_tuite = 1
for tuite in lista_tuites:
    for palavra in tuite:
        #print(tabela_classific_0[0])
        P_0_dado_tuite *= lista_tuites[palavra]  #ou sem a relativa
lista_primevideo_classfic_0

TypeError: list indices must be integers or slices, not str

In [34]:
#Tem que fazer para cada linha 
#CLASSIFICADOR#

#passa por cada tuíte
for tuite in todos_tuites.Treinamento:
    #P(tuite|classificacao)
    P_tuite_dado_classificacao_0 = todos_tuites_relativa[tuite] * 
    

SyntaxError: invalid syntax (<ipython-input-34-aa83ca2c9342>, line 7)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**